In [71]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg16_bn
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from tqdm import tqdm
import wandb

from utils.early_stopping import EarlyStopping

# Set random seed
SEED = 42
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8" # Cublas ting

torch.hub.set_dir('/cs/student/projects1/2021/izaffar/.cache/torch/hub')

In [72]:
DATA_DIR = "/cs/student/projects1/2021/izaffar/FYP/FYP-MUL/data"
# for filename in os.listdir(DATA_DIR):
#     print(filename)

# Use GPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [73]:
config = dict(
    model_arch="brain_mri",
    mask_type="right",
    learning_rate=1e-5,
    epochs=50,
    val_size=20,
    test_size=20,
    batch_size=4,
    k_folds=5,
    criterion="MSE")

In [74]:
def model_pipeline(run_config):
    # make the model, data, and optimization problem
    dataset = CustomDataset(data_dir=DATA_DIR, model_arch=run_config["model_arch"], mask_type=run_config["mask_type"], transform=True)

    # split off test set
    train_val_indices, test_indices = train_test_split(
        list(range(len(dataset))), test_size=(run_config["test_size"]/100), random_state=SEED
    )
    test_dataset = torch.utils.data.Subset(dataset, test_indices)
    test_loader = make_loader(test_dataset, batch_size=run_config["batch_size"])

    # TODO: save best model at checkpoint

    all_train_losses, all_val_losses, all_val_mean_diffs, all_val_std_diffs = [], [], [], []

    # KFold Cross-validation
    kfold = KFold(n_splits=run_config["k_folds"], shuffle=True, random_state=SEED)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(range(len(train_val_indices)))):
        print(f'Fold: {fold+1}')

        # tell wandb to get started
        with wandb.init(project="fyp-mul", entity="imaad-zaffar-ucl", group="experiment_1", name=f"fold_{fold+1}", config=run_config):
            # access all HPs through wandb.config, so logging matches execution!
            config = wandb.config

            # model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
            # print(model)

            # and use them to train the model
            # train(model, train_loader, val_loader, criterion, optimizer, config)

            train_dataset = torch.utils.data.Subset(dataset, train_indices)
            val_dataset = torch.utils.data.Subset(dataset, val_indices)

            train_loader = make_loader(train_dataset, batch_size=config.batch_size)
            val_loader = make_loader(val_dataset, batch_size=config.batch_size)

            model, criterion, optimizer = make(config)
            # print(model)

            # Train the model
            train_losses, val_losses, val_mean_diffs, val_std_diffs = train(model, train_loader, val_loader, criterion, optimizer, fold, config)

            all_train_losses.append(train_losses)
            all_val_losses.append(val_losses)
            all_val_mean_diffs.append(val_mean_diffs)
            all_val_std_diffs.append(val_std_diffs)

            # Log average performance across folds
            # log_average_performance(all_train_losses, all_val_losses, all_val_mean_diffs, all_val_std_diffs)

            # and test its final performance
            test(model, test_loader)

    return model

def log_average_performance(all_train_losses, all_val_losses, all_val_mean_diffs, all_val_std_diffs):
    avg_train_loss = np.mean(all_train_losses, axis=0)
    avg_val_loss = np.mean(all_val_losses, axis=0)
    avg_val_mean_diff = np.mean(all_val_mean_diffs)
    avg_val_std_diff = np.mean(all_val_std_diffs)

    wandb.log({"train_loss_avg": avg_train_loss,
                "val_loss_avg": avg_val_loss,
                "val_mean_diff_avg": avg_val_mean_diff,
                "val_std_diff_avg": avg_val_std_diff})

    print("Average Performance Across Folds:")
    print(f"Avg Train Loss: {avg_train_loss[-1]:.3f}, Avg Val Loss: {avg_val_loss[-1]:.3f}")
    print(f"Avg Val Mean Diff: {avg_val_mean_diff:.3f}, Avg Val Std Diff: {avg_val_std_diff:.3f}")

In [75]:
def make(config):
    # Make the data
    # train, val, test = get_data(val_size=config.val_size, test_size=config.test_size, model_arch=config.model_arch, mask_type=config.mask_type)
    # train_loader = make_loader(train, batch_size=config.batch_size)
    # val_loader = make_loader(val, batch_size=config.batch_size)
    # test_loader = make_loader(test, batch_size=config.batch_size)

    # Make the model
    model = get_model(config.model_arch, DEVICE)

    # Make the loss and optimizer
    if config.criterion == "MSE":
        criterion = nn.MSELoss()
    
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    return model, criterion, optimizer

In [76]:
def convert_pixels_to_mm(length_pixels, images, required_image_width_pixels=320, scale_factor=0.56525):
    image_width_pixels = images.size(2)
    length_mm = length_pixels * (required_image_width_pixels / image_width_pixels) * scale_factor
    return length_mm

def get_length_line_simple(masks):
    batch_size, _, image_width, image_height = masks.size()
    masks = masks.squeeze(1)
    lengths = []

    for i in range(batch_size):
        mask = masks[i]  # Select the mask for the current sample
        
        # Find indices of non-zero elements (where the mask is 1)
        nonzero_indices = torch.nonzero(mask, as_tuple=False)
        
        if len(nonzero_indices) > 0:
            # Compute the bounding box from the non-zero indices
            min_x = nonzero_indices[:, 1].min().item()
            min_y = nonzero_indices[:, 0].min().item()
            max_x = nonzero_indices[:, 1].max().item()
            max_y = nonzero_indices[:, 0].max().item()

            # Calculate the diagonal length of the bounding box
            width = max_x - min_x
            height = max_y - min_y
            diagonal_length = torch.sqrt(torch.tensor(width**2 + height**2, dtype=torch.float32))
            lengths.append(diagonal_length)
        else:
            lengths.append(torch.tensor(0.0))  # If no non-zero elements found, return length 0

    return torch.stack(lengths)

In [77]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, model_arch, mask_type="right", transform=False):
        self.data_dir = data_dir
        self.model_arch = model_arch
        self.transform = transform

        # image dir
        self.image_dir = os.path.join(data_dir, "images_best_slice")
        self.image_files = sorted(os.listdir(self.image_dir))

        # mask dir
        if mask_type == "both":
            self.mask_dir = os.path.join(data_dir, "masks_both")
        elif mask_type == "right":
            self.mask_dir = os.path.join(data_dir, "masks_right")
        self.mask_files = sorted(os.listdir(self.mask_dir))

        assert len(self.image_files) == len(self.mask_files)

    def slice_transform(self, slice_image):
        if self.model_arch == "brain_mri":
            m, s = np.mean(slice_image, axis=(0, 1)), np.std(slice_image, axis=(0, 1))
            preprocess = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.CenterCrop((256, 256)),
                    transforms.Normalize(mean=m, std=s),
                    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]
            )
        input_tensor = preprocess(slice_image)
        return input_tensor

    def mask_transform(self, mask_image):
        if self.model_arch == "brain_mri":
            preprocess = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.CenterCrop((256, 256)),
                ]
            )
        input_tensor = preprocess(mask_image)
        return input_tensor

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image and mask
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        image = np.load(image_path).astype(np.float32)
        mask = np.load(mask_path).astype(np.float32)

        # transform
        if self.transform:
            image = self.slice_transform(image)
            mask = self.mask_transform(mask)

        return image, mask


In [78]:
# def get_data(val_size, test_size, model_arch, mask_type):
#     dataset = CustomDataset(data_dir=DATA_DIR, model_arch=model_arch, mask_type=mask_type, transform=True)
#     train_val_indices, test_indices = train_test_split(
#         list(range(len(dataset))), test_size=(test_size/100), random_state=SEED
#     )
#     train_indices, val_indices = train_test_split(
#         list(range(len(train_val_indices))), test_size=(val_size/(100-test_size)), random_state=SEED
#     )

#     train_dataset = torch.utils.data.Subset(dataset, train_indices)
#     val_dataset = torch.utils.data.Subset(dataset, val_indices)
#     test_dataset = torch.utils.data.Subset(dataset, test_indices)

#     return train_dataset, val_dataset, test_dataset

def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=0)
    return loader

In [79]:
brain_unet_model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
        in_channels=3, out_channels=1, init_features=32, pretrained=True).to(DEVICE)
new_in_channels = 1
modified_encoder1_weight = brain_unet_model.encoder1.enc1conv1.weight.data[:, :new_in_channels, :, :]
brain_unet_model.encoder1.enc1conv1 = nn.Conv2d(new_in_channels, 32, kernel_size=3, padding=1)
brain_unet_model.encoder1.enc1conv1.weight.data = modified_encoder1_weight

# Define your new model by extending the loaded model
class BrainMRIUNetFC(nn.Module):
    def __init__(self, base_model):
        super(BrainMRIUNetFC, self).__init__()
        self.base_model = base_model

        output_features = 256 * 256
        self.final = nn.Sequential(
            nn.Flatten(),
            nn.Linear(output_features, 1)  # Fully connected layer with a single node
        )

    def forward(self, x):
        # Forward pass through the base model
        x = self.base_model(x)

        # Apply your new layer
        x = self.final(x)

        return x

# Load the pre-trained model
def get_model(model_arch, device="cpu"):
    if model_arch == "brain_mri":
        return BrainMRIUNetFC(base_model=brain_unet_model).to(device)
    else:
        raise ValueError("Model type not found")

Using cache found in /cs/student/projects1/2021/izaffar/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [80]:
def train(model, train_loader, val_loader, criterion, optimizer, fold, config):
    train_losses, val_losses = [], []
    val_mean_diffs, val_std_diffs = [], []

    for epoch in range(config.epochs):
        model.train()
        train_loss = 0.0

        for _, (images, masks) in enumerate(train_loader):
            loss = train_batch(images, masks, model, optimizer, criterion)
            train_loss += loss.item() * len(images)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        val_diffs = []

        with torch.no_grad():
            for _, (images, masks) in enumerate(val_loader):
                loss, diff = val_batch(images, masks, model, criterion)
                val_loss += loss.item() * len(images)
                val_diffs.append(diff.item())

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        val_mean_diffs.append(np.mean(val_diffs))
        val_std_diffs.append(np.std(val_diffs))

        # Log metrics
        wandb.log({"epoch": epoch,
                    f"train_loss": train_loss,
                    f"val_loss": val_loss,
                    f"val_mean_diff": val_mean_diffs[-1],
                    f"val_std_diff": val_std_diffs[-1]},
                    step=epoch)
        print(f"Epoch {str(epoch).zfill(3)} - Train: {train_loss:.3f}, Val: {val_loss:.3f}")

    return train_losses, val_losses, val_mean_diffs, val_std_diffs


In [81]:
def train_batch(images, masks, model, optimizer, criterion):
    images, masks = images.to(DEVICE), masks.to(DEVICE)
    target_lengths = get_length_line_simple(masks)
    target_lengths_mm = convert_pixels_to_mm(target_lengths, masks).to(DEVICE)
    
    # Forward pass ➡
    outputs = model(images).squeeze(1)
    loss = criterion(outputs, target_lengths_mm)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

def val_batch(images, masks, model, criterion):
    images, masks = images.to(DEVICE), masks.to(DEVICE)
    target_lengths = get_length_line_simple(masks)
    target_lengths_mm = convert_pixels_to_mm(target_lengths, masks).to(DEVICE)
    
    # Forward pass ➡
    outputs = model(images).squeeze(1)
    val_loss = criterion(outputs, target_lengths_mm)
    val_diff = torch.mean(torch.abs(outputs - target_lengths_mm))

    return val_loss, val_diff

In [82]:
def train_log(train_loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": train_loss}, step=example_ct)
    print(f"Train Loss after {str(example_ct).zfill(5)} examples: {train_loss:.3f}")

def val_log(val_loss, val_diff, example_ct, epoch):
    # Where the magic happens
    wandb.log({"val_loss": val_loss, "val_diff": val_diff}, step=example_ct)
    print(f"Val Loss after {str(example_ct).zfill(5)} examples: {val_loss:.3f}")

In [83]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        diffs = []
        total = 0
        for images, masks in test_loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            targets_length = get_length_line_simple(masks)
            targets_length_mm = convert_pixels_to_mm(targets_length, masks).to(DEVICE)

            outputs = model(images).squeeze(1)
            total += masks.size(0)
            diffs.extend(torch.abs(outputs - targets_length_mm).detach().cpu().numpy())
        
        mean_diff = np.mean(diffs)
        std_diff = np.std(diffs)
        print(f"Metrics for {total} test images - Mean Diff: {mean_diff}mm, Std Diff: {std_diff}mm")

        wandb.log({"mean_diff": np.mean(diffs), "std_diff": np.std(diffs)})

    # Save the model in the exchangeable ONNX format
    # torch.onnx.export(model, images, "model.onnx")
    # wandb.save("model.onnx")

In [84]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config)

Fold: 1


Epoch 000 - Train: 503.346, Val: 527.411
Epoch 001 - Train: 464.153, Val: 436.831
Epoch 002 - Train: 379.911, Val: 353.310
Epoch 003 - Train: 273.977, Val: 254.643
Epoch 004 - Train: 172.369, Val: 165.034
Epoch 005 - Train: 97.077, Val: 93.515
Epoch 006 - Train: 59.035, Val: 54.722
Epoch 007 - Train: 38.132, Val: 38.884
Epoch 008 - Train: 32.716, Val: 31.877
Epoch 009 - Train: 31.092, Val: 27.369
Epoch 010 - Train: 26.370, Val: 31.343
Epoch 011 - Train: 19.337, Val: 32.541
Epoch 012 - Train: 16.364, Val: 31.388
Epoch 013 - Train: 17.573, Val: 34.055
Epoch 014 - Train: 12.213, Val: 31.112
Epoch 015 - Train: 11.096, Val: 33.656
Epoch 016 - Train: 7.912, Val: 34.704
Epoch 017 - Train: 7.127, Val: 35.621
Epoch 018 - Train: 6.407, Val: 36.699
Epoch 019 - Train: 7.086, Val: 36.340
Epoch 020 - Train: 7.244, Val: 37.127
Epoch 021 - Train: 4.394, Val: 35.656
Epoch 022 - Train: 3.566, Val: 35.251
Epoch 023 - Train: 5.707, Val: 35.807
Epoch 024 - Train: 6.369, Val: 33.010
Epoch 025 - Train: 4.624

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
mean_diff,▁
std_diff,▁
train_loss,█▇▆▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mean_diff,██▆▅▃▁▁▂▁▁▁▁▁▂▁▁▂▁▂▂▂▁▁▁▂▁▁▂▂▁▂▂▁▁▁▁▂▂▁▂
val_std_diff,▅▇▆▄▄▅▅█▅▃▆▃▅▂▂▃▄▄▅▄▄▄▂▅▅▃▃▄▄▄▃▂▅▄▁▄▆▆▄▂
epoch,49
mean_diff,3.77372
std_diff,3.30279
train_loss,3.50688


Fold: 2


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
mean_diff,▁
std_diff,▁
train_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mean_diff,█▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_std_diff,▆█▄▅▃▂▁▂▂▁▂▁▁▁▂▂▁▁▂▁▁▁▂▁▂▂▁▁▁▁▁▁▂▃▂▂▁▂▂▁
epoch,49
mean_diff,1.26902
std_diff,1.35261
train_loss,0.60026


Fold: 5


Epoch 000 - Train: 443.582, Val: 347.687
Epoch 001 - Train: 251.415, Val: 193.520
Epoch 002 - Train: 121.103, Val: 85.995
Epoch 003 - Train: 50.762, Val: 35.666
Epoch 004 - Train: 22.772, Val: 16.085
Epoch 005 - Train: 10.915, Val: 8.071
Epoch 006 - Train: 8.101, Val: 4.929
Epoch 007 - Train: 6.639, Val: 4.528
Epoch 008 - Train: 3.734, Val: 2.651
Epoch 009 - Train: 3.057, Val: 2.552
Epoch 010 - Train: 2.392, Val: 1.804
Epoch 011 - Train: 2.469, Val: 1.434
Epoch 012 - Train: 2.504, Val: 2.498
Epoch 013 - Train: 1.528, Val: 1.590
Epoch 014 - Train: 1.955, Val: 1.160
Epoch 015 - Train: 2.124, Val: 1.336
Epoch 016 - Train: 1.374, Val: 1.148
Epoch 017 - Train: 0.832, Val: 0.995
Epoch 018 - Train: 1.656, Val: 1.351
Epoch 019 - Train: 1.534, Val: 1.022
Epoch 020 - Train: 1.732, Val: 0.873
Epoch 021 - Train: 1.355, Val: 1.102
Epoch 022 - Train: 0.956, Val: 1.064
Epoch 023 - Train: 0.806, Val: 1.058
Epoch 024 - Train: 1.082, Val: 1.178
Epoch 025 - Train: 1.043, Val: 1.311
Epoch 026 - Train: 1.5

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
mean_diff,▁
std_diff,▁
train_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mean_diff,█▆▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_std_diff,▆█▄▇▃▃▃▁▂▂▂▁▂▂▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁
epoch,49
mean_diff,0.73427
std_diff,1.44417
train_loss,0.34742
